# Needed Libraries

requires numpy>=1.21.5, pandas>=1.4.2, PIL>=9.0.1, pymatgen>=2.0.0, patchify>=0.2.3

In [1]:
import numpy as np
import pandas as pd
import random
import pymatgen.core as pg
import matplotlib.pyplot as plt
import numpy as np
import struct
import os
import sys
import struct
import PIL as pil
from PIL import Image
from patchify import patchify, unpatchify

C:\Users\User\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\User\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\User\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\User\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Functions

Functions that allow the main MatPrint function to work

In [2]:
#1 MinMax function 
def minmax_normalize(arr):
    # Find the minimum and maximum values in the array
    min_val = min(arr)
    max_val = max(arr)

    # Normalize the array using min-max normalization
    normalized_arr = [(x - min_val) / (max_val - min_val) for x in arr]

    return normalized_arr

In [3]:
#2 double to binary using the 754 IEEE standard for compression
def double_to_binary(num):
    # handleing the NaN value first
    if num == 'NaN':
        return '0111111111111111111111111111111111111111111111111111111111111111'
    else:
        # pack the float into a byte string
        packed = struct.pack('>d', num)
        # unpack the byte string as an unsigned long long
        bits = struct.unpack('>Q', packed)[0]
        # return the binary representation
        return '{:064b}'.format(bits)

In [4]:
#3 binary to float using the 754 IEEE standard for de-compression
def binary_to_double(binary_str):
    # Ensure that the binary string is 64 bits long
    if len(binary_str) != 64:
        raise ValueError('Input binary string must be 64 bits long')
    # Convert the binary string to an integer
    int_value = int(binary_str, 2)
    # Use the 'struct' module to pack the integer as a 64-bit double
    double_value = struct.unpack('d', struct.pack('Q', int_value))[0]
    return double_value


# Import Data

Import the data that is to be graphically represented, note the data as mentioned in the manuscript has to have all the possible features (composition + crystal) that can be generated using the magpie platform, the features must be 623 in total for a single compound. More information about the features generation process can be obtained at: https://wolverton.bitbucket.io/

In [5]:
# import the example file for 58 C compounds obtained from Materials Project Database 
data = pd.read_csv(r'./sample_example.csv')
# drop the last column named 'counter' that is added to only for distinguishing puposes.
#filename is, well, the poscar filename that is used to generate the features, given in the csv file, but it cam ne any appropriate naming that is considered
#those two columns are essential for the next line of code to work, if its changed, please make sure that you change the second line accordingly.
#data_dropped, is essentially all the 623 features without the counter and filename.
data_dropped = data.drop(['counter','filename'], axis = 'columns')

# Main MatPrint Function p.1 - Copmression (Features2Images -- F2I)

The function uses the IEEE 754 and the data at hand to generate the grapical representation from tabular csv files

**READS A CSV FILE GENERATED BY Magpie**

In [49]:
#the MatPrint_F2I is the features to images function, not a lot of arguments are given here in this code, but it is planned that further customization and error handeling will be implmented for this function
#make sure to make an empty directory manually to save the images
def MatPrint_F2I(all_data, cropped_data, directory):
    ones = np.ones((576,64))
    counter = 0
    counter2 = 0
    material_counter = 1
    #cropping the last 48 features (initial 624 features are a MUST)
    data_dropped = cropped_data[cropped_data.iloc[:,:576].columns]
    #looping and binary value generation (initial 624 features are a MUST)
    for m in range(len(data_dropped)):
        nameing = all_data.loc(0)[m][0]
        numpy_data = np.array(data_dropped.loc(0)[m])
        binary = []
        for f in numpy_data:
            binary.append(double_to_binary(f))
        counter2 = 0
        for j in range(len(binary)):
            counter = 0
            for k in binary[j]:
                ones[counter2][counter] = int(k)
                counter = counter + 1
            counter2 = counter2 + 1
        arr1 = np.array(ones, dtype=np.uint8)*255
        #patchify to stitch all the 9 parts of the image together, im sure there is a more optimzed way, but thought this is relatively fas for those ~500 features
        patches = patchify(ones, (64,64), step=64)
        ar1 = np.pad(patches[0,0,:,:], [(0, 128), (0, 128)], mode='constant', constant_values=1)
        ar2 = np.pad(patches[1,0,:,:], [(64, 64), (0, 128)], mode='constant', constant_values=1)
        ar3 = np.pad(patches[2,0,:,:], [(128, 0), (0, 128)], mode='constant', constant_values=1)
        ar4 = np.pad(patches[3,0,:,:], [(0, 128), (64, 64)], mode='constant', constant_values=1)
        ar5 = np.pad(patches[4,0,:,:], [(64, 64), (64, 64)], mode='constant', constant_values=1)
        ar6 = np.pad(patches[5,0,:,:], [(128, 0), (64, 64)], mode='constant', constant_values=1)
        ar7 = np.pad(patches[6,0,:,:], [(0, 128), (128, 0)], mode='constant', constant_values=1)
        ar8 = np.pad(patches[7,0,:,:], [(64, 64), (128, 0)], mode='constant', constant_values=1)
        ar9 = np.pad(patches[8,0,:,:], [(128, 0), (128, 0)], mode='constant', constant_values=1)
        img = Image.fromarray(arr1)
        full_pic = np.ones((192,192))
        full_pic = (((((((((full_pic*ar7)*ar8)*ar9)*ar6)*ar5)*ar3)*ar4)*ar2)*ar1)
        full = np.array(full_pic, dtype=np.uint8)*255
        #transform to image
        img_full = Image.fromarray(full)
        #output to needed directory, I urge to make a test folder for the example given, but this should be changed if is used for other materials
        img_full.save(f'{directory}' + f'{material_counter}_{nameing}' + '.png', dpi = (600,600))
        material_counter = material_counter + 1

# # Main MatPrint Function p.2 - De-compression (Image2Features -- I2F)

The function uses decryptes the MatPrint generated grpahical representation to generate the csv file

**READS A FOLDER OF .PNG IMAGES GENERATED BY FUNCTION F2I**

In [50]:
def MatPrint_I2F(image_directory,output_name):
    #read the preset 576 feature names + filename of this work from the provided 'features.txt' file
    feature_list = []
    with open('./features.txt') as f:
        feature_list = [line.rstrip('\n') for line in f]
    #read only .png files in the specific directory that is needed
    all_files = os.listdir(image_directory)
    png_files=[f for f in all_files if '.png' in f]
    #create a dataframe with the feature names of this work
    feature_dataframe = pd.DataFrame()
    #transfrom the image to csv from the binary data (might not be very optimized)
    for i in png_files:
        data_image = Image.open(f'./{image_directory}/{i}')
        num_image = np.array(np.asarray(data_image)/255,dtype=np.uint8)
        num_feature = num_image[0:192,0:64]
        num_feature2 = num_image[0:192,64:128]
        num_feature3 =num_image[0:192, 128:192]
        num1_2 = np.concatenate((num_feature, num_feature2), axis=0)
        num2_3 =  np.concatenate((num1_2, num_feature3), axis=0)
        features = []
        naming = i
        features.append(naming)
        #creating the dataframe from hte given folder
        for j in num2_3:
            binary = ''.join(str(k) for k in j)
            features.append(binary_to_double(str(binary)))
        feature_dataframe = feature_dataframe.append(pd.Series(features), ignore_index=True)
    feature_dataframe.columns = feature_list
    feature_dataframe.to_csv(f'{output_name}.csv', na_rep='NaN', index=False)

# Using the given example

In [51]:
MatPrint_F2I(data, data_dropped,'./test/')

In [52]:
MatPrint_I2F('test','samples_decompressed')

<ipython-input-50-f1ae11669908>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_dataframe = feature_dataframe.append(pd.Series(features), ignore_index=True)
